In [1]:
import os
import pandas as pd
from PIL import Image as PILImage
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER, TA_LEFT
from reportlab.lib.units import inch, cm
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
import datetime # 用于获取当前时间（如果需要在文件名中使用）

# 添加中文字体支持 - 改用更可靠的方式
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
# 使用内置字体，避免依赖系统字体
from reportlab.pdfbase.cidfonts import UnicodeCIDFont

# 尝试注册中文字体 - 使用ReportLab内置的中文字体
pdfmetrics.registerFont(UnicodeCIDFont('STSong-Light'))
DEFAULT_FONT = 'STSong-Light'  # 使用内置的中文字体
print(f"已注册中文字体: {DEFAULT_FONT}")

print("库导入完成。")

已注册中文字体: STSong-Light
库导入完成。


In [2]:
# --- 提前创建输出文件夹 ---
# 定义要创建的文件夹名称
OUTPUT_FOLDER_NAME = "生成结果/report_pdf"
output_folder_ready = False # 标记输出文件夹是否准备就绪

try:
    # 使用 os.makedirs 创建文件夹。
    # exist_ok=True 参数意味着如果文件夹已经存在，则不会引发错误。
    # 这使得代码更简洁，因为它结合了检查和创建。
    os.makedirs(OUTPUT_FOLDER_NAME, exist_ok=True)
    print(f"输出目录 '{OUTPUT_FOLDER_NAME}' 已确保存在。")
    output_folder_ready = True # 文件夹已存在或已成功创建
except Exception as e:
    # 如果发生权限问题或其他错误导致无法创建文件夹
    print(f"[严重错误] 无法创建或访问输出目录 '{OUTPUT_FOLDER_NAME}': {e}")
    print("请检查程序运行权限或磁盘空间。PDF生成将中止。")

输出目录 '生成结果/report_pdf' 已确保存在。


In [3]:
# 1. 文件路径设置
#    源文件夹路径保持不变
SOURCE_FOLDER = r"生成结果/last_quadrant"  # <--- 修改这里: 指向包含图片和Excel的文件夹路径

#    定义PDF文件名 (可以包含日期等动态信息)
#    current_time_str = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#    PDF_FILENAME = f"combined_report_{current_time_str}.pdf"
PDF_FILENAME = "总结报告.pdf" # <--- 你可以修改这个基础文件名

#    构建完整的输出PDF路径，将其放入上面创建的文件夹中
OUTPUT_PDF = os.path.join(OUTPUT_FOLDER_NAME, PDF_FILENAME)

In [4]:
# 在配置区域添加新图片的文件夹和图片文件模式
FIRST_IMAGE_FOLDERS = [
    "生成结果/matches_analysis",
    "生成结果/defect_analysis", 
    "生成结果/needs_analysis"
]

# 图片文件后缀模式
FIRST_IMAGE_PATTERNS = [
    "功能场景匹配四象限分析.png",
    "综合得分版.png", 
    "综合得分版.png"
]

# 新图片的标题模板 - 请根据需要修改
FIRST_IMAGE_TITLES = [
    "图 A: VOC-功能匹配分析四象限图",  # 第一张附加图片的标题
    "图 B: VOC-缺陷分析四象限图",      # 第二张附加图片的标题
    "图 C: VOC-需求分析四象限图"       # 第三张附加图片的标题
]

# 新图片的说明文字模板 - 请根据需要修改
FIRST_IMAGE_CAPTIONS = [
    """横轴(X)：功能预期度
功能预期度 = 1 - 归一化(功能意外性分数)
纵轴(Y)：场景价值创造度
产品价值创造度 = 归一化((0.4 × 产品场景评价频率) + (0.4 × 产品场景情感强度) + (0.2 × 产品场景描述具体度))
VOC四象限定义
1. 第一象限：体验突破区（低功能预期度-高产品价值创造度）
  - 理论基础：对应"享乐品质"、温暖度感知和愉悦度感知
  - 特征：消费者未预期但产品在特定场景中创造显著价值的功能
  - 战略意义：形成品牌差异化的核心来源，提升用户忠诚度
2. 第二象限：核心体验区（高功能预期度-高产品价值创造度）
  - 理论基础：结合"实用品质"和"享乐品质"的均衡区域
  - 特征：消费者预期且实际产品创造高场景价值的功能
  - 战略意义：品牌核心竞争力，需持续优化提升体验
3. 第三象限：基础功能区（高功能预期度-低产品价值创造度）
  - 理论基础：对应"实用品质"主导区域
  - 特征：消费者预期但产品场景价值有限的基础功能
  - 战略意义：满足市场准入门槛，需保持行业标准水平
4. 第四象限：潜在机会区（低功能预期度-低产品价值创造度）
  - 理论基础：设计驱动创新的潜在领域
  - 特征：当前价值和预期均低但可能通过设计创新提升的功能
  - 战略意义：长期创新方向，探索性开发领域""",

    """缺陷分析说明：
横轴(X)：痛点严重度
痛点严重度 = 归一化(问题影响程度 + 问题频率)
纵轴(Y)：解决价值度
解决价值度 = 归一化(0.5 × 竞争差距 + 0.3 × 用户提及频率 + 0.2 × 解决难度的倒数)
四象限定义
1. 第一象限：关键改进区（高痛点严重度-高解决价值度）
  - 理论基础：期望差距理论和质量管理理论
  - 特征：严重且频繁出现的问题，解决后可显著提升竞争力
  - 战略意义：优先级最高，需立即解决，直接影响产品市场表现
2. 第二象限：体验提升区（低痛点严重度-高解决价值度）
  - 理论基础：卡诺模型中的魅力质量区域
  - 特征：虽不严重但解决后可带来额外满意度的小问题
  - 战略意义：良好的投资回报，可创造差异化竞争优势
3. 第三象限：长期规划区（低痛点严重度-低解决价值度）
  - 理论基础：边际效用递减理论
  - 特征：非关键且影响有限的次要问题
  - 战略意义：可纳入长期改进规划，资源充足时再考虑解决
4. 第四象限：资源权衡区（高痛点严重度-低解决价值度）
  - 理论基础：资源约束理论
  - 特征：较严重但解决难度大或投入产出比低的问题
  - 战略意义：需寻找创新方案或替代方案，避免高投入低回报""",

    """需求分析说明：
横轴(X)：需求普遍度
需求普遍度 = 归一化(需求提及频率 + 目标用户覆盖率)
纵轴(Y)：满足价值度
满足价值度 = 归一化(0.4 × 用户付费意愿 + 0.4 × 竞争差距 + 0.2 × 实现可行性)
四象限定义
1. 第一象限：战略机会区（高需求普遍度-高满足价值度）
  - 理论基础：蓝海战略和价值创新理论
  - 特征：广泛的高价值需求，市场竞争未充分满足
  - 战略意义：核心竞争力构建区，可显著提升市场份额和用户满意度
2. 第二象限：差异化区（低需求普遍度-高满足价值度）
  - 理论基础：利基市场理论和差异化战略
  - 特征：小众但高价值的需求，满足后可带来强烈忠诚度
  - 战略意义：品牌特色构建，吸引特定高价值用户群体
3. 第三象限：观察评估区（低需求普遍度-低满足价值度）
  - 理论基础：选择性创新和资源优化理论
  - 特征：小众且价值有限的需求
  - 战略意义：持续监测是否演变为更大趋势，暂不投入资源
4. 第四象限：基础满足区（高需求普遍度-低满足价值度）
  - 理论基础：卡诺模型中的必备质量和顾客期望理论
  - 特征：普遍但不会带来显著差异化的基础需求
  - 战略意义：行业标准，需要达到但不宜过度投入"""
]

In [5]:
# 2. 图片文件列表 (按照你希望它们出现的顺序)
IMAGE_FILES = [
    "matches_quadrant_analysis.png",  # <--- 修改这里: 第一个图片的文件名
    "pain_point_quadrant_analysis.png",  # <--- 修改这里: 第二个图片的文件名
    "need_quadrant_analysis.png"  # <--- 修改这里: 第三个图片的文件名
]

# 3. 图片标题列表 (与上面的图片文件一一对应)
IMAGE_TITLES = [
    "图 1: 功能场景四象限图",     # <--- 修改这里: 图片1的标题
    "图 2: 用户痛点四象限图",   # <--- 修改这里: 图片2的标题
    "图 3: 未满足需求四象限图"     # <--- 修改这里: 图片3的标题
]

# 4. 图片说明文字模板 (与上面的图片文件一一对应)
IMAGE_CAPTIONS = [
        """横轴(X)：功能-场景创新度
VOC功能意外性 = 从VOC分析获取的功能意外性分数
社媒创新讨论 = 与创新相关变量("个性化定制需求"、"场景功能需求强度"等)的加权影响力
功能-场景创新度 = (0.4 × 归一化(VOC功能意外性)) + (0.6 × 归一化(社媒创新讨论))

纵轴(Y)：场景价值传播影响力
VOC场景价值 = 从VOC分析获取的场景价值创造度
社媒传播影响 = 相关变量在各消费者旅程阶段的加权传播系数
场景价值传播影响力 = (0.3 × 归一化(VOC场景价值)) + (0.7 × 归一化(社媒传播影响))

四象限定义
1. 第一象限：场景价值引领区
  - 特征：高创新度且具有强传播影响力的功能-场景组合
  - 意义：塑造市场趋势，引领消费者行为转变
  - 行动：重点投入产品营销，构建基于场景的品牌叙事
2. 第二象限：价值共鸣区
  - 特征：创新度不高但传播影响力强的功能-场景组合
  - 意义：强化现有消费文化，增强社群凝聚力
  - 行动：鼓励用户生成内容，发展场景化社群营销
3. 第三象限：功能优化区
  - 特征：创新度与传播影响力均较低的功能-场景组合
  - 意义：优化产品基础体验，提升用户满意度
  - 行动：通过用户研究精细化改进，提升场景适配性
4. 第四象限：隐藏价值区
  - 特征：高创新度但当前传播影响力低的功能-场景组合
  - 意义：发掘未来增长点，培育新兴市场需求
  - 行动：通过教育市场提升认知，构建先发优势""",  
    
    """横轴(X)：痛点影响评分
痛点影响评分 = (0.3 × 归一化痛点严重度) + (0.3 × 归一化情感强度) + (0.2 × 归一化特异性) + (0.2 × ln(1 + 频率))

纵轴(Y)：社媒传播指数
痛点对应变量 = 识别与痛点相关的社媒变量集合（社交媒体洞察与VOC洞察的交集）
变量传播系数 = 回归分析中变量对传播的标准化影响系数（绝对值）
社媒传播指数 = ∑(变量传播系数i × 痛点-变量关联强度i) / ∑痛点-变量关联强度i

痛点-消费者洞察影响四象限定义
1. 第一象限：高痛点需解决区
  - 特征：高痛点影响 + 高社媒传播
  - 意义：严重痛点且易引发负面传播，威胁品牌形象
  - 行动：立即分配资源解决，启动产品更改方案
2. 第二象限：市场消耗区
  - 特征：低痛点影响 + 高社媒传播
  - 意义：实际影响有限但引发广泛讨论的问题
  - 行动：透明沟通并教育市场，制定明确改进时间表
3. 第三象限：后台改进区
  - 特征：低痛点影响 + 低社媒传播
  - 意义：影响有限且不引发广泛讨论的问题
  - 行动：纳入常规产品迭代计划，无需特别关注
4. 第四象限：隐性风险区
  - 特征：高痛点影响 + 低社媒传播
  - 意义：严重但未被广泛讨论的问题，潜在风险高
  - 行动：主动解决并预防可能的口碑危机，关注特定用户群体""", 
    
    """横轴(X)：需求价值综合评分
VOC需求得分 = (0.3 × 归一化重要性) + (0.2 × 归一化情感强度) + (0.3 × 归一化特异性) + (0.2 × ln(1 + 频率))
需求价值综合评分 = (0.6 × VOC需求得分) + (0.4 × 相关变量传播系数加权平均)

纵轴(Y)：消费决策传播影响力
消费者旅程阶段权重 = {"认知": 0.2, "考虑": 0.3, "决策": 0.5}（仅示例）
变量决策加权系数 = 变量传播系数 × 消费者旅程阶段权重[变量所属阶段]
消费决策传播影响力 = ∑(变量决策加权系数i × 需求-变量关联强度i) / ∑需求-变量关联强度i
其中，需求-变量关联强度i可以由预训练模型综合语义相似度给出。

四象限定义
1. 第一象限：核心传播机会区
  - 特征：高需求价值 + 高消费决策传播影响力
  - 意义：高价值且在社交媒体上具有强大传播力和决策推动力的需求
  - 行动：作为产品核心卖点和营销传播重点，强化消费者决策旅程中的存在感
2. 第二象限：市场教育区
  - 特征：低需求价值 + 高消费决策传播影响力
  - 意义：价值相对较低但具有良好传播性的需求，可作为引流话题
  - 行动：利用其传播优势引导消费者认知，同时将关注转向高价值功能
3. 第三象限：低关注潜力区
  - 特征：低需求价值 + 低消费决策传播影响力
  - 意义：价值有限且难以激发市场讨论的需求
  - 行动：低优先级处理，或作为辅助功能点平衡产品结构
4. 第四象限：潜在差异化区
  - 特征：高需求价值 + 低消费决策传播影响力
  - 意义：高价值但当前市场讨论度不高的需求，具有差异化潜力
  - 行动：主动开发并积极传播，可成为品牌差异化的核心来源""", 
]


In [6]:
# 5. Excel 文件列表 (按照你希望它们出现的顺序)
EXCEL_FILES = [
    "建议表1-表达-流量关系表.xlsx", # <--- 修改这里: 第一个Excel文件名
    "建议表2-消费者旅程不同阶段的产品表达关注表.xlsx", # <--- 修改这里: 第二个Excel文件名
    "建议表3-品类功能利益点情感利益点分析表.xlsx",     # <--- 修改这里: 第三个Excel文件名
    "建议表4-产品改进建议.xlsx",     # <--- 修改这里: 第四个Excel文件名
    "建议表5-情感利益点改进建议.xlsx",       # <--- 修改这里: 第五个Excel文件名
    "建议表6-改进优先级排序.xlsx"   # <--- 修改这里: 第六个Excel文件名
]
# 6. PDF 样式和布局设置
PAGE_WIDTH, PAGE_HEIGHT = A4
MAX_CONTENT_WIDTH = PAGE_WIDTH - 2 * inch # 内容（图片、表格）最大宽度

print("配置区域加载完成。")
print(f"源文件目录: {SOURCE_FOLDER}")
print(f"目标输出PDF: {OUTPUT_PDF}") # 确认最终输出路径

配置区域加载完成。
源文件目录: 生成结果/last_quadrant
目标输出PDF: 生成结果/report_pdf\总结报告.pdf


In [7]:
import os
import pandas as pd
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER, TA_LEFT
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from PIL import Image as PILImage # Use a different alias to avoid conflict

# # --- 全局设置 ---
# # 假设你已经在代码的其他地方定义了这些常量
# # 请确保将 'path/to/your/font.ttf' 替换为实际的中文字体文件路径
# try:
#     # 注册中文字体 (例如：思源黑体)
#     # 请确保你有一个名为 'SourceHanSansCN-Regular.otf' 的字体文件，
#     # 或者替换为你实际使用的字体文件名和路径
#     DEFAULT_FONT_PATH = 'SourceHanSansCN-Regular.otf' # <-- 修改为你字体的实际路径
#     if os.path.exists(DEFAULT_FONT_PATH):
#         pdfmetrics.registerFont(TTFont('SourceHanSansCN-Regular', DEFAULT_FONT_PATH))
#         DEFAULT_FONT = 'SourceHanSansCN-Regular'
#         print(f"字体 '{DEFAULT_FONT}' 加载成功。")
#     else:
#         # 如果找不到特定字体，回退到 reportlab 可能自带或系统默认的字体
#         # 这可能不支持中文，需要用户确保环境中有合适的字体
#         print(f"[警告] 指定字体文件未找到: {DEFAULT_FONT_PATH}。将尝试使用默认字体，可能不支持中文。")
#         DEFAULT_FONT = 'Helvetica' # 或者其他备选字体
# except Exception as e:
#     print(f"[严重错误] 加载字体时出错: {e}。将尝试使用默认字体，可能不支持中文。")
#     DEFAULT_FONT = 'Helvetica'

# 页面尺寸和边距 (A4)
PAGE_WIDTH, PAGE_HEIGHT = (595.27, 841.89) # A4 size in points
LEFT_MARGIN = inch
RIGHT_MARGIN = inch
TOP_MARGIN = inch
BOTTOM_MARGIN = inch
MAX_CONTENT_WIDTH = PAGE_WIDTH - LEFT_MARGIN - RIGHT_MARGIN

# 假设这些图片相关的列表也在别处定义了
FIRST_IMAGE_FOLDERS = [] # Example: ['path/to/folder1']
FIRST_IMAGE_PATTERNS = [] # Example: ['pattern1']
FIRST_IMAGE_TITLES = []   # Example: ['Title 1']
FIRST_IMAGE_CAPTIONS = [] # Example: ['Caption 1']


def create_pdf_jupyter(output_filename, source_folder,
                       image_files, image_titles, image_captions,
                       excel_files):
    """
    在Jupyter环境中生成包含图片和Excel表格的PDF文件。
    图片在前（带标题和说明），Excel表格在后。
    """
    doc = SimpleDocTemplate(output_filename, pagesize=(PAGE_WIDTH, PAGE_HEIGHT),
                            leftMargin=LEFT_MARGIN, rightMargin=RIGHT_MARGIN,
                            topMargin=TOP_MARGIN, bottomMargin=BOTTOM_MARGIN)
    styles = getSampleStyleSheet()
    story = []

    # --- 自定义样式，现在使用中文字体 ---
    image_title_style = ParagraphStyle(name='ImageTitleStyle',
                               parent=styles['Heading3'],
                               alignment=TA_CENTER,
                               spaceAfter=6,
                               fontName=DEFAULT_FONT)  # 使用中文字体

    caption_style = ParagraphStyle(name='CaptionStyle',
                           parent=styles['Normal'],  # 改用Normal，确保可读性
                           alignment=TA_LEFT,
                           fontSize=9,  # 稍微减小字体大小以适应大量文本
                           spaceAfter=4,  # 减小段落间距
                           leading=12,  # 增加行间距以提高可读性
                           fontName=DEFAULT_FONT,  # 使用中文字体
                           leftIndent=5,
                           rightIndent=5)

    excel_title_style = ParagraphStyle(name='ExcelTitleStyle',
                               parent=styles['Heading2'],
                               alignment=TA_LEFT,
                               spaceAfter=6,
                               fontName=DEFAULT_FONT)  # 使用中文字体

    # 覆盖默认样式，为其添加中文字体
    normal_style = ParagraphStyle(name='CustomNormal',
                          parent=styles['Normal'],
                          fontName=DEFAULT_FONT)  # 使用中文字体

    error_style = ParagraphStyle(name='ErrorStyle',
                         parent=normal_style,
                         textColor=colors.red)

    print(f"\n--- 开始准备PDF内容 ---")

    # --- 0. 首先处理新增的三张图片 ---
    print("\n--- 处理新增图片部分 ---")

    found_FIRST_images = []

    # 确保列表长度一致，避免 zip 提前终止
    min_len = min(len(FIRST_IMAGE_FOLDERS), len(FIRST_IMAGE_PATTERNS), len(FIRST_IMAGE_TITLES), len(FIRST_IMAGE_CAPTIONS))

    for i in range(min_len):
        folder = FIRST_IMAGE_FOLDERS[i]
        pattern = FIRST_IMAGE_PATTERNS[i]
        title = FIRST_IMAGE_TITLES[i]
        caption = FIRST_IMAGE_CAPTIONS[i]

        print(f"  搜索文件夹 {folder} 中包含模式 '{pattern}' 的图片...")

        if not os.path.isdir(folder):
            print(f"  [警告] 文件夹不存在: {folder}")
            continue

        # 获取文件夹中所有文件
        try:
            all_files = os.listdir(folder)
            matching_files = [f for f in all_files if pattern in f]  # 使用 'in' 而不是 'endswith'

            if not matching_files:
                print(f"  [警告] 在 {folder} 中没有找到包含 '{pattern}' 的图片")
                continue

            # 如果有多个匹配文件，可以根据需要选择，这里选第一个
            if len(matching_files) > 1:
                print(f"  [注意] 在 {folder} 中找到多个匹配文件: {matching_files}")
                # 可以添加逻辑选择最新的文件等，这里简单选第一个
                print(f"  将使用第一个文件: {matching_files[0]}")

            img_path = os.path.join(folder, matching_files[0])
            found_FIRST_images.append((img_path, title, caption))
            print(f"  已找到图片: {img_path}")
        except Exception as e:
             print(f"  [错误] 搜索或处理文件夹 {folder} 时出错: {e}")


    # 添加找到的新增图片到PDF
    for img_path, title_text, caption_text in found_FIRST_images:
        try:
            img_title = Paragraph(title_text, image_title_style)
            story.append(img_title)

            with PILImage.open(img_path) as pil_img:
                img_width, img_height = pil_img.size
                aspect_ratio = img_height / float(img_width) if img_width > 0 else 1
                display_width = MAX_CONTENT_WIDTH
                display_height = display_width * aspect_ratio

                # 防止图片过高导致跨页问题，可以设置一个最大高度
                max_img_height = PAGE_HEIGHT * 0.7 # 例如，不超过页面高度的70%
                if display_height > max_img_height:
                    display_height = max_img_height
                    display_width = display_height / aspect_ratio

            img = Image(img_path, width=display_width, height=display_height)
            story.append(img)
            story.append(Spacer(1, 0.1*inch))

            # 处理图片说明，保持精确换行
            caption_paragraphs_raw = caption_text.split('\n\n') # 按空行分割段落

            for paragraph_raw in caption_paragraphs_raw:
                lines = paragraph_raw.split('\n') # 按换行符分割行
                para_text = '<br/>'.join(line for line in lines if line.strip()) # 用<br/>连接非空行
                if para_text:
                    story.append(Paragraph(para_text, caption_style))
                    story.append(Spacer(1, 0.05*inch)) # 段落间的细微间距

            story.append(Spacer(1, 0.3*inch)) # 图片及其说明后的总间距
            print(f"  成功添加新增图片: {os.path.basename(img_path)}")

        except Exception as e:
            print(f"  [错误] 处理新增图片 {img_path} 时出错: {e}")
            story.append(Paragraph(f"[错误] 处理新增图片 {os.path.basename(img_path)} 时出错: {e}", error_style))
            story.append(Spacer(1, 0.2*inch))

    # 如果添加了新增图片，插入一个分页符
    if found_FIRST_images:
        story.append(PageBreak())
        print("  在新增图片后添加分页符")

    # --- 1. 处理常规图片 ---
    print("\n--- 处理常规图片部分 ---")
    if len(image_files) != len(image_titles) or len(image_files) != len(image_captions):
        print("[警告] 图片文件、标题、说明的数量不一致，请检查配置！")
        # 决定如何处理：是截断到最短长度还是跳过？这里假设截断
        min_len_img = min(len(image_files), len(image_titles), len(image_captions))
        image_files = image_files[:min_len_img]
        image_titles = image_titles[:min_len_img]
        image_captions = image_captions[:min_len_img]


    for i, img_filename in enumerate(image_files):
        print(f"  处理图片 {i+1}/{len(image_files)}: {img_filename}")
        img_path = os.path.join(source_folder, img_filename)
        title_text = image_titles[i]
        caption_text = image_captions[i]

        if not os.path.exists(img_path):
            print(f"  [错误] 图片文件未找到: {img_path}")
            story.append(Paragraph(f"[错误] 图片文件未找到: {img_filename}", error_style))
            story.append(Spacer(1, 0.2*inch))
            continue
        try:
            img_title = Paragraph(title_text, image_title_style)
            story.append(img_title)

            with PILImage.open(img_path) as pil_img:
                img_width, img_height = pil_img.size
                aspect_ratio = img_height / float(img_width) if img_width > 0 else 1
                display_width = MAX_CONTENT_WIDTH
                display_height = display_width * aspect_ratio

                # 防止图片过高
                max_img_height = PAGE_HEIGHT * 0.7
                if display_height > max_img_height:
                    display_height = max_img_height
                    display_width = display_height / aspect_ratio

            img = Image(img_path, width=display_width, height=display_height)
            story.append(img)
            story.append(Spacer(1, 0.1*inch))

            # 处理图片说明，保持精确换行 (与新增图片部分逻辑相同)
            caption_paragraphs_raw = caption_text.split('\n\n')

            for paragraph_raw in caption_paragraphs_raw:
                lines = paragraph_raw.split('\n')
                para_text = '<br/>'.join(line for line in lines if line.strip())
                if para_text:
                    story.append(Paragraph(para_text, caption_style))
                    story.append(Spacer(1, 0.05*inch))

            story.append(Spacer(1, 0.3*inch))
            print(f"  成功添加图片: {img_filename}")
        except Exception as e:
            print(f"  [错误] 处理图片 {img_filename} 时出错: {e}")
            story.append(Paragraph(f"[错误] 处理图片 {img_filename} 时出错: {e}", error_style))
            story.append(Spacer(1, 0.2*inch))

    # --- 2. 处理Excel表格 ---
    print("\n--- 处理Excel表格部分 ---")
    # 可选：在图片和表格之间加分页符
    if image_files or found_FIRST_images: # 如果有任何图片被添加
         story.append(PageBreak())
         print("  在图片和表格之间添加分页符")

    for i, excel_filename in enumerate(excel_files):
        print(f"  处理Excel {i+1}/{len(excel_files)}: {excel_filename}")
        excel_path = os.path.join(source_folder, excel_filename)
        if not os.path.exists(excel_path):
            print(f"  [错误] Excel文件未找到: {excel_path}")
            story.append(Paragraph(f"[错误] Excel文件未找到: {excel_filename}", error_style))
            story.append(Spacer(1, 0.2*inch))
            continue
        try:
            df = pd.read_excel(excel_path, sheet_name=0)
            df = df.fillna('') # 用空字符串填充 NaN
            table_title_text = f"表格 {i+1}: {os.path.splitext(excel_filename)[0]}"
            excel_title = Paragraph(table_title_text, excel_title_style)
            story.append(excel_title)
            story.append(Spacer(1, 0.1*inch)) # 标题和表格间的间距

            headers = df.columns.astype(str).tolist()
            data_values = df.values.tolist() # 直接获取列表的列表

            # 创建表格数据，将所有文本转换为Paragraph对象以支持中文和自动换行
            header_style = ParagraphStyle('Header',
                                 parent=styles['Normal'], # 基于Normal
                                 fontName=DEFAULT_FONT,
                                 fontSize=8,
                                 alignment=TA_CENTER,
                                 textColor=colors.whitesmoke,
                                 leading=10) # 行间距

            cell_style = ParagraphStyle('Cell',
                                parent=styles['Normal'], # 基于Normal
                                fontName=DEFAULT_FONT,
                                fontSize=8,
                                leading=10,  # 行间距
                                spaceBefore=2, # 段落前间距
                                spaceAfter=2) # 段落后间距

            formatted_headers = [Paragraph(h, header_style) for h in headers]
            formatted_data = [formatted_headers]

            # --- 核心修改：处理单元格内容 ---
            for row_idx, row in enumerate(data_values):
                formatted_row = []
                for col_idx, cell in enumerate(row):
                    try:
                        # 1. 转换为字符串
                        cell_content = str(cell)
                        # 2. 替换 <br> 和 <br/> 为 ReportLab 的换行符 <br/>
                        #    ReportLab 的 Paragraph 更倾向于识别 <br/>
                        processed_content = cell_content.replace('<br>', '<br/>').replace('<br />', '<br/>')
                        # 3. 创建 Paragraph 对象
                        para = Paragraph(processed_content, cell_style)
                        formatted_row.append(para)
                    except Exception as cell_e:
                        print(f"    [警告] 处理单元格 ({row_idx+1}, {col_idx+1}) 内容时出错: {cell_e}")
                        print(f"      原始内容: {cell!r}")
                        # 出错时用原始字符串创建，可能显示不佳但避免程序中断
                        formatted_row.append(Paragraph(str(cell), cell_style))
                formatted_data.append(formatted_row)
            # --- 修改结束 ---

            # 使用处理后的Paragraph对象创建表格
            table = Table(formatted_data, repeatRows=1) # repeatRows=1 使表头每页重复

            table_style_config = [
                ('BACKGROUND', (0, 0), (-1, 0), colors.darkblue),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('ALIGN', (0, 0), (-1, 0), 'CENTER'),      # 表头居中
                ('VALIGN', (0, 0), (-1, 0), 'MIDDLE'),     # 表头垂直居中
                ('FONTNAME', (0, 0), (-1, 0), DEFAULT_FONT),
                ('BOTTOMPADDING', (0, 0), (-1, 0), 6),    # 表头底部填充
                ('TOPPADDING', (0, 0), (-1, 0), 6),        # 表头顶部填充

                ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
                ('ALIGN', (0, 1), (-1, -1), 'LEFT'),       # 内容左对齐
                ('VALIGN', (0, 1), (-1, -1), 'TOP'),       # 内容顶部对齐，配合Paragraph换行
                ('FONTNAME', (0, 1), (-1, -1), DEFAULT_FONT),
                ('BOTTOMPADDING', (0, 1), (-1, -1), 4),
                ('TOPPADDING', (0, 1), (-1, -1), 4),
                ('GRID', (0, 0), (-1, -1), 0.5, colors.grey), # 添加网格线
                # WORDWRAP 对 Paragraph 对象作用不大，因为Paragraph内部处理换行
                # ('WORDWRAP', (0, 1), (-1, -1), 'CJK'), # 可选，用于处理中日韩字符断行
            ]
            table.setStyle(TableStyle(table_style_config))


            # --- 自动调整列宽 (保持之前的逻辑) ---
            num_cols = len(formatted_data[0])
            col_widths = []
            # 尝试更智能地估算列宽，给包含换行的列更多宽度
            available_width = MAX_CONTENT_WIDTH

            # 初步估算：基于内容长度和换行符数量
            estimated_widths = []
            for j in range(num_cols):
                max_chars = 0
                max_lines = 1
                # 检查表头
                header_text = headers[j]
                header_len = sum(2 if '\u4e00' <= c <= '\u9fff' else 1 for c in header_text)
                max_chars = max(max_chars, header_len)

                # 检查数据单元格
                for r in range(len(data_values)):
                    cell_text = str(data_values[r][j])
                    lines = cell_text.replace('<br/>', '\n').replace('<br>', '\n').split('\n')
                    max_lines = max(max_lines, len(lines))
                    for line in lines:
                        line_len = sum(2 if '\u4e00' <= c <= '\u9fff' else 1 for c in line)
                        max_chars = max(max_chars, line_len)

                # 基础宽度 + 基于内容和行数的调整
                # 这里的系数需要根据实际效果调整
                est_width = (max_chars * 6) + (max_lines * 5) # 简单估算，单位 pt
                estimated_widths.append(est_width)

            # 按比例分配可用宽度
            total_estimated_width = sum(estimated_widths)
            if total_estimated_width > 0:
                scale_factor = available_width / total_estimated_width
                col_widths = [w * scale_factor for w in estimated_widths]
            else: # 如果所有列都为空
                col_widths = [available_width / num_cols] * num_cols

            # 添加最小列宽限制
            min_col_width_pt = 0.5 * inch # 最小宽度
            col_widths = [max(w, min_col_width_pt) for w in col_widths]

            # 如果调整后总宽度超过限制，再次缩放
            current_total_width = sum(col_widths)
            if current_total_width > available_width:
                final_scale = available_width / current_total_width
                col_widths = [w * final_scale for w in col_widths]
                print(f"    表格 '{excel_filename}' 宽度超过页面限制，已按比例缩小列宽。")


            table._argW = col_widths # 设置计算好的列宽

            story.append(table)
            story.append(Spacer(1, 0.3*inch)) # 表格后的间距
            print(f"  成功添加Excel表格: {excel_filename}")

        except Exception as e:
            import traceback
            print(f"  [严重错误] 处理Excel文件 {excel_filename} 时发生意外错误: {e}")
            print(traceback.format_exc()) # 打印详细的回溯信息
            story.append(Paragraph(f"[错误] 处理Excel文件 {excel_filename} 时出错: {e}", error_style))
            story.append(Spacer(1, 0.2*inch))

    # --- 生成PDF文件 ---
    try:
        print("\n--- 开始构建PDF文档 ---")
        doc.build(story)
        print(f"--- PDF文件已成功保存到: {output_filename} ---")
        return True # 表示成功
    except Exception as e:
        import traceback
        print(f"[严重错误] 生成PDF时发生错误: {e}")
        print(traceback.format_exc()) # 打印详细的回溯信息
        return False # 表示失败

In [8]:
# --- 主执行逻辑 (直接在Notebook单元格中运行) ---

pdf_generated_successfully = False

# 检查输出文件夹是否已成功准备好 (在代码开头处理)
if output_folder_ready:
    # 检查源文件夹是否存在
    if not os.path.isdir(SOURCE_FOLDER):
        print(f"[错误] 源文件夹不存在: {SOURCE_FOLDER}")
        print("请在代码顶部的配置区域修改 SOURCE_FOLDER 变量为正确的路径。")
    else:
        # 源文件夹存在，输出文件夹也准备好了，可以调用PDF生成函数
        print("\n准备就绪，开始调用PDF生成函数...")
        pdf_generated_successfully = create_pdf_jupyter(
            OUTPUT_PDF,
            SOURCE_FOLDER,
            IMAGE_FILES,
            IMAGE_TITLES,
            IMAGE_CAPTIONS,
            EXCEL_FILES
        )
else:
    # 如果输出文件夹未能准备好（例如创建失败）
    print("\n由于输出目录未能准备好，PDF生成任务已中止。")


# --- 最终状态 ---
if pdf_generated_successfully:
    print("\n🎉 PDF生成任务完成！")
else:
    # 如果是因为文件夹问题中止，上面已经打印了信息
    # 如果是PDF生成过程中失败，这里会打印
    if output_folder_ready: # 只有在文件夹没问题时才可能是生成过程出错
      print("\n❌ PDF生成任务遇到问题，请检查上面的错误信息。")


准备就绪，开始调用PDF生成函数...

--- 开始准备PDF内容 ---

--- 处理新增图片部分 ---

--- 处理常规图片部分 ---
  处理图片 1/3: matches_quadrant_analysis.png
  成功添加图片: matches_quadrant_analysis.png
  处理图片 2/3: pain_point_quadrant_analysis.png
  成功添加图片: pain_point_quadrant_analysis.png
  处理图片 3/3: need_quadrant_analysis.png
  成功添加图片: need_quadrant_analysis.png

--- 处理Excel表格部分 ---
  在图片和表格之间添加分页符
  处理Excel 1/6: 建议表1-表达-流量关系表.xlsx


  成功添加Excel表格: 建议表1-表达-流量关系表.xlsx
  处理Excel 2/6: 建议表2-消费者旅程不同阶段的产品表达关注表.xlsx
    表格 '建议表2-消费者旅程不同阶段的产品表达关注表.xlsx' 宽度超过页面限制，已按比例缩小列宽。
  成功添加Excel表格: 建议表2-消费者旅程不同阶段的产品表达关注表.xlsx
  处理Excel 3/6: 建议表3-品类功能利益点情感利益点分析表.xlsx
    表格 '建议表3-品类功能利益点情感利益点分析表.xlsx' 宽度超过页面限制，已按比例缩小列宽。
  成功添加Excel表格: 建议表3-品类功能利益点情感利益点分析表.xlsx
  处理Excel 4/6: 建议表4-产品改进建议.xlsx


    表格 '建议表4-产品改进建议.xlsx' 宽度超过页面限制，已按比例缩小列宽。
  成功添加Excel表格: 建议表4-产品改进建议.xlsx
  处理Excel 5/6: 建议表5-情感利益点改进建议.xlsx


    表格 '建议表5-情感利益点改进建议.xlsx' 宽度超过页面限制，已按比例缩小列宽。
  成功添加Excel表格: 建议表5-情感利益点改进建议.xlsx
  处理Excel 6/6: 建议表6-改进优先级排序.xlsx
    表格 '建议表6-改进优先级排序.xlsx' 宽度超过页面限制，已按比例缩小列宽。
  成功添加Excel表格: 建议表6-改进优先级排序.xlsx

--- 开始构建PDF文档 ---


--- PDF文件已成功保存到: 生成结果/report_pdf\总结报告.pdf ---

🎉 PDF生成任务完成！
